# Estimar esforço em desenvolvimento de software com redes neurais profundas

Desenvolvimento do Trabalho de Graduação


## Dependências

- Python 3.6
- numpy >= 1.15.4
- pandas >= 0.23.4
- matplotlib >= 2.2.3

In [30]:
from glob import glob
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras

## Constantes globais de configurações

In [6]:
caminho_dados = 'dados/cocomo/'

## Carregamento e pre-processamento dos dados

In [26]:
def carregar_tudo(caminho_dados):
    dados = {}
    for arquivo in glob(caminho_dados+'/*.csv'):
        ticker = arquivo.split('/')[-1].split('.')[0]
        tabela = pd.read_csv(arquivo, skipinitialspace=True)
        dados[ticker] = tabela
    return dados

dados = carregar_tudo(caminho_dados)
dados.keys()

dict_keys(['cocomo_nasa2', 'cocomo_sdr', 'cocomo81', 'cocomo_nasa'])

In [27]:
dados['cocomo_sdr'].keys()

Index(['ProjectID', 'PREC', 'FLEX', 'RESL', 'TEAM', 'PMAT', 'RELY', 'DATA',
       'CPLX', 'RUSE', 'DOCU', 'TIME', 'STOR', 'PVOL', 'ACAP', 'PCAP', 'PCON',
       'AEXP', 'PEXP', 'LTEX', 'TOOL', 'SITE', 'SCED', 'LOC', 'ACTUAL EFFORT'],
      dtype='object')

In [23]:
sorted(['RELY', ' DATA', ' CPLX', ' TIME', ' STOR', ' VIRT', ' TURN', ' ACAP',
       ' AEXP', ' PCAP', ' VEXP', ' LEXP', ' MODP', ' TOOL', ' SCED', ' LOC'])

[' ACAP',
 ' AEXP',
 ' CPLX',
 ' DATA',
 ' LEXP',
 ' LOC',
 ' MODP',
 ' PCAP',
 ' SCED',
 ' STOR',
 ' TIME',
 ' TOOL',
 ' TURN',
 ' VEXP',
 ' VIRT',
 'RELY']

In [28]:
sorted(['PREC', 'FLEX', 'RESL', 'TEAM', 'PMAT', 'RELY', 'DATA',
       'CPLX', 'RUSE', 'DOCU', 'TIME', 'STOR', 'PVOL', 'ACAP', 'PCAP', 'PCON',
       'AEXP', 'PEXP', 'LTEX', 'TOOL', 'SITE', 'SCED', 'LOC', 'ACTUAL EFFORT'])

['ACAP',
 'ACTUAL EFFORT',
 'AEXP',
 'CPLX',
 'DATA',
 'DOCU',
 'FLEX',
 'LOC',
 'LTEX',
 'PCAP',
 'PCON',
 'PEXP',
 'PMAT',
 'PREC',
 'PVOL',
 'RELY',
 'RESL',
 'RUSE',
 'SCED',
 'SITE',
 'STOR',
 'TEAM',
 'TIME',
 'TOOL']

In [9]:
dados['cocomo81']

,id,prec,flex,resl,team,pmat,rely,data,cplx,ruse,...,apex,plex,ltex,tool,site,sced,kloc,effort,defects,months
0,1,h,h,h,vh,vl,l,vh,vl,n,...,l,l,n,vl,h,n,113,2040.0,13027,38.4
1,2,h,h,h,vh,vl,l,vh,l,n,...,h,h,h,vl,h,n,293,1600.0,25229,48.6
2,3,h,h,h,vh,n,n,vh,l,n,...,vh,h,h,l,h,n,132,243.0,3694,28.7
3,4,h,h,h,vh,vl,vl,vh,vl,n,...,h,n,h,vl,h,n,60,240.0,5688,28.0
4,5,h,h,h,vh,vl,l,l,n,n,...,n,h,h,vl,h,n,16,33.0,970,14.3
5,6,h,h,h,vh,vl,vl,n,l,n,...,n,h,h,vl,h,n,4,43.0,553,11.6
6,7,h,h,h,vh,n,vl,n,n,n,...,n,h,h,l,h,n,6.9,8.0,350,10.3
7,8,h,h,h,vh,vl,h,l,vh,n,...,h,vl,vl,vl,h,l,22,1075.0,3511,24.5
8,9,h,h,h,vh,n,h,l,vh,n,...,n,l,l,vl,h,n,30,423.0,1989,24.1
9,10,h,h,h,vh,l,vh,l,vh,n,...,vh,h,n,vl,h,n,29,321.0,1496,23.2


In [10]:
dados['cocomo_nasa']

,RELY,DATA,CPLX,TIME,STOR,VIRT,TURN,ACAP,AEXP,PCAP,VEXP,LEXP,MODP,TOOL,SCED,LOC,ACT_EFFORT
0,Nominal,High,Very_High,Nominal,Nominal,Low,Nominal,High,Nominal,Very_High,Low,Nominal,High,Nominal,Low,70.0,278.0
1,Very_High,High,High,Very_High,Very_High,Nominal,Nominal,Very_High,Very_High,Very_High,Nominal,High,High,High,Low,227.0,1181.0
2,Nominal,High,High,Very_High,High,Low,High,High,Nominal,High,Low,High,High,Nominal,Low,177.9,1248.0
3,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,115.8,480.0
4,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,29.5,120.0
5,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,19.7,60.0
6,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,66.6,300.0
7,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,5.5,18.0
8,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,10.4,50.0
9,High,Low,High,Nominal,Nominal,Low,Low,Nominal,Nominal,Nominal,Nominal,High,High,Nominal,Low,14.0,60.0


In [11]:
dados['cocomo_nasa2']

,recordnumber,projectname,cat2,forg,center,year,mode,rely,data,cplx,...,acap,aexp,pcap,vexp,lexp,modp,tool,sced,equivphyskloc,act_effort
0,1,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,25.90,117.6
1,2,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,24.60,117.6
2,3,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,7.70,31.2
3,4,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,8.20,36.0
4,5,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,9.70,25.2
5,6,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,2.20,8.4
6,7,de,avionicsmonitoring,g,2,1979,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,3.50,10.8
7,8,erb,avionicsmonitoring,g,2,1982,semidetached,h,l,h,...,n,n,n,n,h,h,n,l,66.60,352.8
8,9,gal,missionplanning,g,1,1980,semidetached,h,l,h,...,h,h,h,n,h,h,h,n,7.50,72.0
9,10,gal,missionplanning,g,1,1980,semidetached,n,l,h,...,h,vh,vh,n,h,n,n,n,20.00,72.0


In [12]:
dados['cocomo_sdr']

,ProjectID,PREC,FLEX,RESL,TEAM,PMAT,RELY,DATA,CPLX,RUSE,...,PCAP,PCON,AEXP,PEXP,LTEX,TOOL,SITE,SCED,LOC,ACTUAL EFFORT
0,1,Very High,Nominal,Low,High,Very Low,Nominal,Nominal,Low,Low,...,Very High,Very High,Very High,High,High,High,Low,Nominal,3000,1.2
1,2,Very High,Nominal,Low,Very High,Very Low,High,Nominal,Low,Nominal,...,High,Very High,Very High,Very High,Very High,High,Very Low,High,2000,2.0
2,3,Extra High,Nominal,Low,High,Very Low,High,Very High,Low,Nominal,...,High,Very High,Very High,Very High,Very High,High,Very Low,High,4250,4.5
3,4,Extra High,High,High,High,Nominal,High,High,Very High,Low,...,High,Very Low,High,Very High,High,Nominal,High,High,10000,3.0
4,5,Nominal,Nominal,Low,High,Nominal,Low,Nominal,Very High,Nominal,...,High,High,Nominal,Nominal,Nominal,Nominal,High,Low,15000,4.0
5,6,Very High,Low,Extra High,Extra High,High,Low,Low,Nominal,Low,...,Very High,High,Nominal,High,High,Very Low,Extra High,Low,40530,22.0
6,7,Extra High,Very High,Extra High,Nominal,High,Low,Low,Low,Nominal,...,Very High,Very High,Nominal,Very High,Very High,Very Low,High,Low,4050,2.0
7,8,Nominal,Very Low,Very High,Nominal,Low,Low,Very High,Very Low,High,...,High,Very High,Nominal,Nominal,Nominal,Nominal,Very High,Nominal,31845,5.0
8,9,Very Low,Very Low,Nominal,Very Low,Low,Very Low,Nominal,Low,Very High,...,High,Very High,High,High,High,Nominal,High,Nominal,114280,18.0
9,10,Nominal,Very Low,Very High,Low,Low,Very Low,High,Nominal,Very High,...,Very High,Very High,High,Very High,High,Nominal,Nominal,Low,23106,4.0


## Criação do modelo COCOMO neural

In [35]:
params = pd.read_csv('dados/params-cocomo2000.csv', skipinitialspace=True, index_col=0)
params

,Symbol,VL,L,N,H,VH,XH
Driver,,,,,,,
PREC,SF1,6.20,4.96,3.72,2.48,1.24,0.00
FLEX,SF2,5.07,4.05,3.04,2.03,1.01,0.00
RESL,SF3,7.07,5.65,4.24,2.83,1.41,0.00
TEAM,SF4,5.48,4.38,3.29,2.19,1.10,0.00
PMAT,SF5,7.80,6.24,4.68,3.12,1.56,0.00
RELY,EM1,0.82,0.92,1.00,1.10,1.26,NaN
DATA,EM2,NaN,0.90,1.00,1.14,1.28,NaN
CPLX,EM3,0.73,0.87,1.00,1.17,1.34,1.74
RUSE,EM4,NaN,0.95,1.00,1.07,1.15,1.24
